In [36]:
from selenium import webdriver
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import time
import pandas as pd
import requests
import bs4
import re

def craw_ettoday(hours=2):
    browser = webdriver.Chrome(executable_path='./chromedriver')
    browser.get("https://www.ettoday.net/news/news-list.htm")

    two_hours_ago = datetime.now() - timedelta(hours = hours)
    print(two_hours_ago)
    two_hours_ago_time = two_hours_ago.strftime("%Y/%m/%d %H:%M")
    print("兩小時前時間：", two_hours_ago_time)
    last_height = browser.execute_script("return document.body.scrollHeight")

    go=True

    while go:
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        html_source = browser.page_source
        soup = BeautifulSoup(html_source, "lxml")

        new_height = browser.execute_script("return document.body.scrollHeight")

        #已經到頁面底部
        if new_height == last_height:
            print("已經到頁面最底部，程序停止")
            break

        last_height = new_height

        for d in soup.find(class_="part_list_2").find_all('h3'):
            #已經超出兩小時
            if datetime.strptime(d.find(class_="date").text, '%Y/%m/%d %H:%M') < two_hours_ago:
                print("已經超出兩個小時，程序停止")
                go = False
                break

            else:
                print("目前畫面最下方文章的日期時間為：",d.find_all(class_="date")[-1].text)
    html_source = browser.page_source
    soup = BeautifulSoup(html_source, "lxml")

    news ={"日期時間":[],"標題":[],"連結":[]}

    for d in soup.find(class_="part_list_2").find_all('h3'):
        if two_hours_ago_time in d.find(class_="date") :
            pass
        else:
            print(d.find(class_="date").text, d.find_all('a')[-1].text)
            news["日期時間"].append(d.find(class_="date").text)
            news["標題"].append(d.find_all('a')[-1].text)
            news["連結"].append("https://www.ettoday.net" + d.find_all('a')[-1]["href"])

    browser.close()  #很重要喔記得要關掉瀏覽器．不然要手動關掉
    df_news=pd.DataFrame(news)
    df_content = pd.DataFrame(columns=['title','url','content'])
    for idx in df_news.index:
        標題 = df_news.loc[idx,'標題']
        連結 = df_news.loc[idx,'連結']
        soup = bs4.BeautifulSoup(requests.get(連結).text,"html.parser")
        內文 = re.sub('[\001\002\003\004\005\006\007\x08\x09\x0a\x0b\x0c\x0d\x0e\x0f\x10\x11\x12\x13\x14\x15\x16\x17\x18\x19\x1a]+','',str(soup.select('div.story>p')))
        df_content.loc[idx,'title'] = 標題
        df_content.loc[idx,'url'] = 連結
        df_content.loc[idx,'content'] = 內文
    return df_content

In [37]:
res = craw_ettoday()
res

C:\Users\Administrator\AppData\Local\Temp\ipykernel_16432\1898898022.py:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(executable_path='./chromedriver')


2023-01-21 22:26:37.338945
兩小時前時間： 2023/01/21 22:26
已經到頁面最底部，程序停止
2023/01/22 00:25 「戰爭沒有贏家」　馬英九除夕祈願：希望兩岸儘快回到10年前
2023/01/22 00:20 TOYOTA今年要推「入門跨界7人座休旅」！台灣未來有望導入
2023/01/22 00:18 快訊／賴清德、馬英九法鼓山撞鐘　致詞談「兩岸和平」互別苗頭
2023/01/22 00:15 台南祀典大天后宮　恢復停辦3年除夕開廟門搶頭香祈平安
2023/01/22 00:15 年度10大事件／格林開季前揮拳揍普爾　勇士王朝恐解體
2023/01/22 00:11 宵夜首選！網推爆「10大追劇泡麵」榜單來了　冠軍最夯口味是它
2023/01/22 00:10 快訊／大龍峒保安宮國運籤開出：等待明朝鼓浪息，還君自在任遷移
2023/01/22 00:09 34家投注站開百萬紅包！台中2間前年「生出」億萬富翁　強運擋不住
2023/01/22 00:04 獲靈鷲山普仁獎學金　桃園國中小太陽黃浩翔當場捐出展大愛
2023/01/22 00:03 3年魔咒催婚必分！女王常收粉絲私訊這問題　告誡：強求沒好結局
2023/01/22 00:01 主人病倒！　暖心汪「整晚床邊守候」狂度估　聽咳嗽聲秒靠近關心
2023/01/22 00:00 蔡阿嘎號稱「Youtuber界長青樹」！其實他們也都拍片超過10年了
2023/01/21 23:53 湧蓮寺抽出國運籤「貴人一指笑顏開」！ETtoday祝讀者鴻兔大展
2023/01/21 23:45 藍寶堅尼「新一代大牛」確認今年發表！採油電引擎　告別純油V12
2023/01/21 23:41 2022台籃10大事件回顧　高雄海神風起雲湧奪T1職籃冠軍
2023/01/21 23:40 新港奉天宮恢復搶頭香！「滑壘哥」衝門縫驚險奪第一　19秒影曝光
2023/01/21 23:40 2022台籃10大事件回顧　富邦勇士2連霸王朝登場
2023/01/21 23:38 氣溫驟降易釀心血管疾病好發　北榮桃園分院籲正視心血管健康
2023/01/21 23:30 王子練半年當菸槍「殺青就戒了！」　演藝圈好寶寶：歷任女友都放心
2023/01/21 23:29 過年「最雷3食物」體脂狂飆！爽

,title,url,content
0,「戰爭沒有贏家」 馬英九除夕祈願：希望兩岸儘快回到10年前,https://www.ettoday.net/news/20230122/2427469.htm,"[<p><img alt=""▲▼賴清德、馬英九出席2023法鼓山 #108 for All ..."
1,TOYOTA今年要推「入門跨界7人座休旅」！台灣未來有望導入,https://www.ettoday.net/news/20230122/2427430.htm,"[<p><img alt=""▲TOYOTA將在亞太區推出小改款新Yaris與Yaris Cr..."
2,快訊／賴清德、馬英九法鼓山撞鐘 致詞談「兩岸和平」互別苗頭,https://www.ettoday.net/news/20230122/2427451.htm,"[<p><img alt=""▲▼賴清德、馬英九出席2023法鼓山 #108 for All ..."
3,台南祀典大天后宮 恢復停辦3年除夕開廟門搶頭香祈平安,https://www.ettoday.net/news/20230122/2427467.htm,"[<p><img alt=""▲因新冠肺炎疫情停辦3年的台南祀典大天后宮開廟門插頭香活動，21..."
4,年度10大事件／格林開季前揮拳揍普爾 勇士王朝恐解體,https://www.ettoday.net/news/20230122/2423572.htm,"[<p><img alt=""▲▼ 格林（Draymond Green）揮拳揍普爾（Jorda..."
...,...,...,...
95,資產滾到8位數！三寶奶爸「簡單1招」買0050 靠股息養全家,https://www.ettoday.net/news/20230121/2426664.htm,"[<p><img alt=""全職奶爸黃紹博，現在光靠0050的股息就足夠應付孩子的生活開銷。..."
96,24歲海巡下士轉戰房仲 成交送iPhone14吸讚...年薪輾壓前東家,https://www.ettoday.net/news/20230121/2423902.htm,"[<p><img alt=""▲▼ 。（圖／Woody提供）"" hei..."
97,馬英九盼兔年「迎來大地回春」 「馬式春聯」5年變化看這,https://www.ettoday.net/news/20230121/2418013.htm,"[<p><img alt=""▲馬英九兔年春聯 。（圖／馬英九基金會提供）"" height=..."
98,饋下巴好爽！邊境聽到握手「瞬間無力」抬不動 媽笑：50肩發作？,https://www.ettoday.net/news/20230121/2414941.htm,"[<p><iframe allowfullscreen=""true"" frameborder..."
